# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 5 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study06'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 100-200 --> 288:
	{'snapshot_id': 'AAACDeaHreqL9UT5EpoXe7di9FEoWuG0'}
Iteration 2, 0-100 --> 316:
	{'snapshot_id': 'AAACDn4slC2Big4gbBZu3i8BFHopXpjj'}
Iteration 3, 200-300 --> 307:
	{'snapshot_id': 'AAACD0a9jzAOnM+Kq9wKjuCJxE790nti'}

Phase 2: Chunks of 10

Iteration 1, 290-300 --> 29:
	{'snapshot_id': 'AAACEHYaiiP1Msz2nXqowdlABjAVaXur'}
Iteration 2, 260-270 --> 163:
	{'snapshot_id': 'AAACEefDCepYO0QTrjS720oUPrqt/WJv'}
Iteration 3, 270-280 --> 208:
	{'snapshot_id': 'AAACEtmK56/RrCYvxgAha63XAgXSRhl9'}
Iteration 4, 270-280 --> 313:
	{'snapshot_id': 'AAACE4wh61d/P6BvA5N0HzZYy8AKNfOj'}
Iteration 5, 330-340 --> 337:
	{'snapshot_id': 'AAACFBygwTXgBPg5AzLzcJIu9M5vBGEC'}
Iteration 6, 200-210 --> 253:
	{'snapshot_id': 'AAACFRyk4o/GKpmfs9iADG3b7RMkP7CT'}
Iteration 7, 350-360 --> 3:
	{'snapshot_id': 'AAACFnHNSJlWLWtFON1qqrIZ+ScVCach'}
Iteration 8, 140-150 --> 132:
	{'snapshot_id': 'AAACFw7M8EQXjF1zE8F5qeGty9Bpejaj'}
Iteration 9, 30-40 --> 144:
	{'snapshot_id'

HTTP Error for PUT to https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 11, 230-240 --> 109:
	{'snapshot_id': 'AAACGugx6bzVaNIJ07X5FRJ4b5l3bLwI'}
ERROR, 390-391 --> 350:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 13, 250-260 --> 306:
	{'snapshot_id': 'AAACG+j7z9jpRCKaopxvy9oxgMcWvwN9'}
Iteration 14, 340-350 --> 241:
	{'snapshot_id': 'AAACHNykur3FbM3ttp8CVwa1XmeohWLA'}
Iteration 15, 300-310 --> 161:
	{'snapshot_id': 'AAACHfoEhx/hsWXi0YKtuEXhCSKlWVKg'}
Iteration 16, 370-380 --> 126:
	{'snapshot_id': 'AAACHuNJXGNIn6fTJag5fWCXHtXKW5jk'}
Iteration 17, 140-150 --> 315:
	{'snapshot_id': 'AAACH1VhORUrjxyFf12WCGcbW7iySfA1'}
Iteration 18, 190-200 --> 250:
	{'snapshot_id': 'AAACIB/Wv1JAUl4XA6eFIK8zLI/deune'}
Iteration 19, 0-10 --> 115:
	{'snapshot_id': 'AAACIQRYqs4MJT6De20oy1TUSZ2HamvR'}
Iteration 20, 370-380 --> 298:
	{'snapshot_id': 'AAACIt0uVmU/KMPSFeoUcuZOOfBnD8fs'}
Iteration 21, 310-320 --> 140:
	{'snapshot_id': 'AAACI85CwWdJ

HTTP Error for PUT to https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 22, 300-310 --> 61:
	{'snapshot_id': 'AAACJFtQkJ/Kfb6puz17DNtxTvnYY5OX'}
ERROR, 390-391 --> 241:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 24, 260-270 --> 329:
	{'snapshot_id': 'AAACJc3yAKDWlsCjH+KPqkCyvjbO2lLc'}
Iteration 25, 150-160 --> 363:
	{'snapshot_id': 'AAACJiwACKN0w1frBN7n4Yu0acQzW24t'}
Iteration 26, 210-220 --> 50:
	{'snapshot_id': 'AAACJ+fdhZ2fRba973N1dRACHy6+5hZn'}
Iteration 27, 30-40 --> 212:
	{'snapshot_id': 'AAACKKb5FAyrpgnp4ObJykgROVQWtSir'}
Iteration 28, 120-130 --> 338:
	{'snapshot_id': 'AAACKZBuWmM74x/BKDahewQwm4cQpC85'}
Iteration 29, 340-350 --> 51:
	{'snapshot_id': 'AAACKqyiYPC3n4dgLtf0otCb9V5c/H9T'}
Iteration 30, 210-220 --> 64:
	{'snapshot_id': 'AAACK0Y4NpX+ICvTw3LgibMc8JqXs77a'}
Iteration 31, 0-10 --> 345:
	{'snapshot_id': 'AAACLJzhGQl2za/Rm9CxU6DcGZevbw/W'}
Iteration 32, 320-330 --> 96:
	{'snapshot_id': 'AAACLWyFJMmZQ2CIKd9

HTTP Error for PUT to https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 36, 320-330 --> 373:
	{'snapshot_id': 'AAACMclbTnAAnXD9NFXn2cZJn+kiMidr'}
Iteration 37, 160-170 --> 324:
	{'snapshot_id': 'AAACMroPmTxipIF3dwefjf/Vr/UO/DLv'}
ERROR, 390-391 --> 132:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 39, 80-90 --> 382:
	{'snapshot_id': 'AAACM7qwCh1e39omjuoFQdbu8ZGARf+g'}

Phase 3: Chunks of 5, 1/2

Iteration 1, 180-185 --> 76:
	{'snapshot_id': 'AAACNODl7lA22JuTZAcVZROciwjgPEQC'}
Iteration 2, 340-345 --> 116:
	{'snapshot_id': 'AAACNZkWfwnX7jC5i9L9JRftf2AECvWn'}
Iteration 3, 360-365 --> 382:
	{'snapshot_id': 'AAACNmOlgneWh1uS+UoOl8sgepnEMMnO'}
Iteration 4, 30-35 --> 370:
	{'snapshot_id': 'AAACN1ms2PGrHXxao+QV6OUG/qZ5OTC5'}
Iteration 5, 100-105 --> 240:
	{'snapshot_id': 'AAACOLmpSBYA6jlcCn51xW7G+2xYwjBQ'}
Iteration 6, 260-265 --> 92:
	{'snapshot_id': 'AAACOTlS6UGZr8z+c+JexmYtRQo2xllY'}
Iteration 7, 15-20 --> 168:
	{'snapshot_id